In [1]:
// This is all just boilerplate, nothing to see here :).
import * as tslab from 'tslab';
import rimraf from 'rimraf';
try {
    rimraf.sync("./default")
} catch(e) {
    // do nothing here
}

# Using Tupelo
In this exmaple we'll walk through a few concepts of using Tupelo using the tupelo-wasm-sdk (TODO: link). There is also a go sdk evailable (TODO: link). 

This javascript SDK can be used in both the browser and nodejs. 

Let's do a bit of setup work:

In [2]:
import { ChainTree, Community, EcdsaKey, setDataTransaction } from 'tupelo-wasm-sdk'

## Networking & Community

The Community object is how the local client talks to the network. You'll need one of these to play your transactions. There's a default (currently the testnet) provided for you.

In [3]:
const community = await Community.getDefault()

## ChainTrees

ChainTrees are the very first thing to understand about Tupelo. They are used to represent every object in the system. They can hold arbitrary data and are owned by one or more owners (represented by public/private key pairs).

A ChainTree is a tree of current state and all the updates that lead up to that state (a blockchain). A blank ChainTree has a root node, an an empty tree, and an empty chain. This is what we call the genesis state. It's a fresh ChainTree that only lives on this machine.

[![](https://mermaid.ink/img/eyJjb2RlIjoiY2xhc3NEaWFncmFtXG5cdFJvb3QgPHwtLSBDaGFpblxuXHRSb290IDx8LS0gVHJlZVxuXHRjbGFzcyBSb290IHtcbiAgICBpZDogZGlkOnR1cGVsbzp4eHhcblx0fVxuXHRjbGFzcyBDaGFpbntcblx0XHRlbXB0eVxuXHR9XG5cdGNsYXNzIFRyZWV7XG5cdFx0ZW1wdHlcblx0fVxuXHRcdFx0XHRcdCIsIm1lcm1haWQiOnsidGhlbWUiOiJkZWZhdWx0In0sInVwZGF0ZUVkaXRvciI6ZmFsc2V9)](https://mermaid-js.github.io/mermaid-live-editor/#/edit/eyJjb2RlIjoiY2xhc3NEaWFncmFtXG5cdFJvb3QgPHwtLSBDaGFpblxuXHRSb290IDx8LS0gVHJlZVxuXHRjbGFzcyBSb290IHtcbiAgICBpZDogZGlkOnR1cGVsbzp4eHhcblx0fVxuXHRjbGFzcyBDaGFpbntcblx0XHRlbXB0eVxuXHR9XG5cdGNsYXNzIFRyZWV7XG5cdFx0ZW1wdHlcblx0fVxuXHRcdFx0XHRcdCIsIm1lcm1haWQiOnsidGhlbWUiOiJkZWZhdWx0In0sInVwZGF0ZUVkaXRvciI6ZmFsc2V9)

Let's create a ChainTree now.

First we create a key:

In [4]:
const key = await EcdsaKey.generate()

Then we create a brand new ChainTree:

In [5]:
const tree = await ChainTree.newEmptyTree(community.blockservice, key)

An empty tree doesn't have much data in it, but it already has a did and a root node. You can traverse a ChainTree using either `resolve` or `resolveData`. We'll get to resolveData in a second.

In [6]:
console.log("id: ", await tree.id())
console.log(await tree.resolve("/"))

id:  did:tupelo:0x60DdeA46407382bC9340B20326111Ac8D21Fc95E
{
  remainderPath: [],
  value: {
    id: 'did:tupelo:0x60DdeA46407382bC9340B20326111Ac8D21Fc95E',
    tree: CID(bafyreigbtj4x7ip5legnfznufuopl4sg4knzc2cof6duas4b3q2fy6swua),
    chain: CID(bafyreigbtj4x7ip5legnfznufuopl4sg4knzc2cof6duas4b3q2fy6swua)
  }
}


Take a look at the results above. You see that a ChainTree has a unique ID called a DID. This DID is based on the key that creates the ChainTree (in this case the `key` we created above).

When you see a value prefaced by `CID` that means it's a *link* to another node. You can resolve through those links using the `resolve` method on a tree. As we said above, this is a genesis ChainTree so there isn't much in there:

In [7]:
console.log("tree: ", (await tree.resolve("/tree")).value)
console.log("chain: ", (await tree.resolve("/chain")).value)

tree:  {}
chain:  {}


### Transactions

You interact with ChainTrees by playing transactions. The simplest transaction is the setData transaction.

In [8]:
await community.playTransactions(tree, [setDataTransaction("/foo", "bar")])
console.log("tree: ", (await tree.resolve("/tree")).value)
console.log("tree/data: ", (await tree.resolve("/tree/data")).value)
console.log("chain: ", (await tree.resolve("/chain")).value)

tree:  {
  data: CID(bafyreiblaotetvwobe7cu2uqvnddr6ew2q3cu75qsoweulzku2egca4dxq)
}
tree/data:  { foo: 'bar' }
chain:  {
  end: CID(bafyreiagfqchwf6nypsnv62inum24wd2ipq5gb5vvy52j76bihork7cv5e)
}


Graphically:

[![](https://mermaid.ink/img/eyJjb2RlIjoiY2xhc3NEaWFncmFtXG4gIFJvb3QgLi4gVHJlZVxuICBUcmVlIC4uIERhdGFcbiAgUm9vdCAuLiBDaGFpblxuICBjbGFzcyBSb290IHtcbiAgICBpZDogZGlkOnR1cGVsbzp4eHhcbiAgICBDSUQgZGF0YVxuICAgIENJRCBjaGFpblxuICB9XG4gIGNsYXNzIFRyZWUge1xuICAgIENJRCBkYXRhXG4gIH1cbiAgY2xhc3MgRGF0YSB7XG4gICAgZm9vOiBcImJhclwiXG4gIH1cbiAgXG4gIGNsYXNzIENoYWluIHtcbiAgICBDSUQgZW5kXG4gIH1cbiIsIm1lcm1haWQiOnsidGhlbWUiOiJkZWZhdWx0In0sInVwZGF0ZUVkaXRvciI6ZmFsc2V9)](https://mermaid-js.github.io/mermaid-live-editor/#/edit/eyJjb2RlIjoiY2xhc3NEaWFncmFtXG4gIFJvb3QgLi4gVHJlZVxuICBUcmVlIC4uIERhdGFcbiAgUm9vdCAuLiBDaGFpblxuICBjbGFzcyBSb290IHtcbiAgICBpZDogZGlkOnR1cGVsbzp4eHhcbiAgICBDSUQgZGF0YVxuICAgIENJRCBjaGFpblxuICB9XG4gIGNsYXNzIFRyZWUge1xuICAgIENJRCBkYXRhXG4gIH1cbiAgY2xhc3MgRGF0YSB7XG4gICAgZm9vOiBcImJhclwiXG4gIH1cbiAgXG4gIGNsYXNzIENoYWluIHtcbiAgICBDSUQgZW5kXG4gIH1cbiIsIm1lcm1haWQiOnsidGhlbWUiOiJkZWZhdWx0In0sInVwZGF0ZUVkaXRvciI6ZmFsc2V9)

You see how the ChainTree now has values in both the tree section `tree/data` and has a new entry in the `chain` section? That's your transaction! We have also now registered this ChainTree with tupelo and so no one in the world can ever modify your ChainTree except you. You lucky duck.

Let's introduce that `resolveData` method here. That's just a convenience method to start your path in `tree/data` rather than the root. We set the path "/tree/data/foo" above - you can easily get the same data back:

In [9]:
const response = await tree.resolveData("/foo")
console.log("foo: ", response.value)

foo:  bar
